# AI as Judge

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM as a judge to evaluate LLM outputs. The evaluation can be based on any criteria. G-Eval is implemented by a library called [DeepEval](https://deepeval.com/) which includes a broader set of tests.


In [1]:
%load_ext dotenv
%dotenv ../../05_src/.secrets

In [2]:
from openai import OpenAI
import os

document_folder = "../../05_src/documents/"
blue_cross_file = "the_blue_cross.txt"
file_path = os.path.join(document_folder, blue_cross_file)

with open(file_path, "r", encoding="utf-8") as f:
    blue_cross_text = f.read()

In [3]:
instructions = "You are an helpful assistant that summarizes works of fiction with a quirky and bubbly approach."
PROMPT = """
    Summarize the following story in at most four paragraphs. Please include all key characters and plot points.
    <story>
    {story}
    </story>
    In addition to the summary, add an introduction paragraph where you greet the reader and a conclusion where you share an opinion about the story.
"""

In [4]:
import os
client = OpenAI(base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1', 
                api_key='any value',
                default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')})
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=1.2
)

In [5]:
response.output_text

'Hello there, fabulous reader! 🌟✨ Are you ready to dive into the delightful twists and turns of "The Blue Cross" by G. K. Chesterton? Join me as we unravel a tale filled with intrigue, cleverness, and just the right amount of quirky humor!\n\nIn this captivating story, we follow the astute Detective Aristide Valentin, head of the Paris police, as he embarks on a mission to catch the infamous criminal Flambeau, known for his spectacular heists and uncanny knack for deception. Disguised as an unassuming fellow traveler, Valentin arrives at Harwich, on an escapade from Brussels to London amid the chaos of the Eucharistic Congress. Determined to thwart Flambeau, who is keen on stealing a valuable silver cross with sapphires, Valentin faces the daunting challenge of identifying his tall adversary among the sea of other elders, market gardeners, and priests – including a seemingly bumbling little priest named Father Brown.\n\nAs the story unfolds, we follow Valentin’s clever, albeit unconven

# Answer Relevancy

The answer relevancy metric evaluates how relevant the actual output of the LLM app is compared to the provided input. This metric is self-explaining in the sense that the output includes a reason for the metric score.

The metric is calculated as:

$$
AnswerRelevancy=\frac{NumberRelevantStatements}{TotalStatements}
$$

Reference: [Answer Relevancy](https://deepeval.com/docs/metrics-answer-relevancy). 

In [6]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval.models import GPTModel

model = GPTModel(
    model="gpt-4o-mini",
    temperature=0,
    # api_key='any value',
    default_headers={"x-api-key": os.getenv('API_GATEWAY_KEY')},
    base_url='https://k7uffyg03f.execute-api.us-east-1.amazonaws.com/prod/openai/v1',
)

metric = AnswerRelevancyMetric(
    threshold=0.7,
    include_reason=True,
    model=model,
    
)

test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text,
    
)

In [7]:
metric.measure(test_case)

Output()

0.9523809523809523

In [8]:
from IPython.display import display, Markdown
display(Markdown(f'**Score**: {metric.score}'))
display(Markdown(f'**Reason**: {metric.reason}'))

**Score**: 0.9523809523809523

**Reason**: The score is 0.95 because while the summary effectively captures the key characters and plot points of 'The Blue Cross', it incorrectly mentions Irene Sinnat, who is not a character in the story. This minor error prevents a perfect score, but the overall quality of the summary remains high, demonstrating a strong understanding of the narrative.

# Other Metrics

Other useful metric functions include:

+ [Faithfulness](https://deepeval.com/docs/metrics-faithfulness): evaluates whether the `actual_output` factually aligns with the contents of  `retrieval_context`. 
+ [Contextual Precision](https://deepeval.com/docs/metrics-contextual-precision): evaluates whether nodes in your `retrieval_context` that are relevant to the given input are ranked higher than irrelevant ones. 
+ [Contextual Recall](https://deepeval.com/docs/metrics-contextual-recall): evaluates the extent of which the retrieval_context aligns with the expected_output. 
+ [Contextual Relevancy](https://deepeval.com/docs/metrics-contextual-relevancy): evaluates the overall relevance of the information presented in your retrieval_context for a given input. 

# G-Eval

[G-Eval](https://deepeval.com/docs/metrics-llm-evals) is a framework that uses LLM-as-a-judge with chain-of-thoughts (CoT) to evaluate LLM outputs based on ANY custom criteria. The G-Eval metric is the most versatile type of metric deepeval offers.

In [9]:
instructions = "You are an helpful assistant that specializes in works of fiction."
PROMPT = """
    Based on the story below, answer the question provided.
    <story>
    {story}
    </story>
    <question>
    Who is the main antagonist in the story and what motivates their actions?
    </question>
"""

In [10]:
response = client.responses.create(
    model="gpt-4o-mini",
    instructions=instructions,
    input=[
        {"role": "user", 
         "content": PROMPT.format(story=blue_cross_text)}
    ],
    temperature=0.7
)

In [11]:
response.output_text

'The main antagonist in "The Blue Cross" is Flambeau, a notorious criminal known for his cleverness and physical prowess. His motivation in the story is to steal a valuable silver cross with sapphires, which is being transported by Father Brown, a naive priest from Essex. Flambeau\'s actions are driven by his desire for wealth and the thrill of committing clever heists, showcasing his cunning nature as a master thief.\n\nThroughout the story, Flambeau employs various disguises and tricks, believing he can outsmart those around him. His confidence in his skills and his past successes in crime motivate him to attempt the theft, ultimately leading to a confrontation with Father Brown, who surprises Flambeau with his own intelligence and resourcefulness.'

## Evaluation Criteria

The most straightforward way to establish a metric is by using a single criteria.

In [12]:
from deepeval.metrics import GEval
from deepeval.test_case import LLMTestCaseParams

correctness_metric = GEval(
    name="Correctness",
    criteria="Determine whether the actual output is factually correct based on the context.",
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

In [13]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.8946407384123445, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, which is to steal the silver cross. It effectively highlights Flambeau's cunning nature and his confidence in outsmarting others, aligning well with the context of the story. The explanation of his motivations and the confrontation with Father Brown is also well-articulated, demonstrating a strong understanding of the narrative., error: None)

For test case:

  - input: 
    Based on the story below, answer the question provided.
    <story>
    The Blue Cross
By G. K. Chesterton 
From the book "The innocence of Father Brown"

Between the silver ribbon of morning and the green glittering ribbon ofsea, the boat touched Harwich and let loose a swarm of folk like flies,among whom the man we must follow was by no means conspicuous--nor

✓ Evaluation completed 🎉! (time taken: 6.2s | token cost: 0.00162945 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='test_case_0', success=True, metrics_data=[MetricData(name='Correctness [GEval]', threshold=0.5, success=True, score=0.8946407384123445, reason="The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, which is to steal the silver cross. It effectively highlights Flambeau's cunning nature and his confidence in outsmarting others, aligning well with the context of the story. The explanation of his motivations and the confrontation with Father Brown is also well-articulated, demonstrating a strong understanding of the narrative.", strict_mode=False, evaluation_model='gpt-4o-mini', error=None, evaluation_cost=0.00162945, verbose_logs='Criteria:\nDetermine whether the actual output is factually correct based on the context. \n \nEvaluation Steps:\n[\n    "1. Identify the context of the input to establish the factual basis for evaluation.",\n    "2. Analyze the actual output to determin

## Evaluation Steps 

G-Eval is flexible in many ways: notice that we can establish an evaluation criteria or a set of evaluation steps, that can help in guiding the model to follow specific steps to perform the evaluation.

In [14]:
...

correctness_metric = GEval(
    name="Correctness",
    evaluation_steps=[
        "Check whether the facts in 'actual output' contradicts any facts in 'input'",
        "You should also heavily penalize omission of detail",
        "Vague language, or contradicting OPINIONS, are not OK"
    ],
    evaluation_params=[LLMTestCaseParams.INPUT, LLMTestCaseParams.ACTUAL_OUTPUT],
    model=model,
)

In [15]:
test_case = LLMTestCase(
    input=PROMPT.format(story=blue_cross_text),
    actual_output=response.output_text
)
result = evaluate(test_cases=[test_case], metrics=[correctness_metric])

✨ You're running DeepEval's latest Correctness [GEval] Metric! (using gpt-4o-mini, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ✅ Correctness [GEval] (score: 0.8188148453123636, threshold: 0.5, strict: False, evaluation model: gpt-4o-mini, reason: The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, which aligns well with the story's details. It mentions his desire to steal the silver cross and highlights his cleverness and confidence, reflecting his character as depicted in the narrative. However, it could benefit from more specific details about Flambeau's past crimes or his relationship with Father Brown to enhance the depth of the analysis., error: None)

For test case:

  - input: 
    Based on the story below, answer the question provided.
    <story>
    The Blue Cross
By G. K. Chesterton 
From the book "The innocence of Father Brown"

Between the silver ribbon of morning and the green glittering ribbon ofsea, the boat touched Harwich and let loose a swarm of folk like flies,among whom the man we must follow was by no m

✓ Evaluation completed 🎉! (time taken: 4.93s | token cost: 0.0015481499999999999 USD)
» Test Results (1 total tests):
   » Pass Rate: 100.0% | Passed: 1 | Failed: 0

 ================================================================================ 

» What to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

In [16]:
result.model_dump()

{'test_results': [{'name': 'test_case_0',
   'success': True,
   'metrics_data': [{'name': 'Correctness [GEval]',
     'threshold': 0.5,
     'success': True,
     'score': 0.8188148453123636,
     'reason': "The response accurately identifies Flambeau as the main antagonist and provides a clear motivation for his actions, which aligns well with the story's details. It mentions his desire to steal the silver cross and highlights his cleverness and confidence, reflecting his character as depicted in the narrative. However, it could benefit from more specific details about Flambeau's past crimes or his relationship with Father Brown to enhance the depth of the analysis.",
     'strict_mode': False,
     'evaluation_model': 'gpt-4o-mini',
     'error': None,
     'evaluation_cost': 0.0015481499999999999,
     'verbose_logs': 'Criteria:\nNone \n \nEvaluation Steps:\n[\n    "Check whether the facts in \'actual output\' contradicts any facts in \'input\'",\n    "You should also heavily penal